In [1]:
# Librerie
import os
import pandas as pd
import numpy as np
import scipy as sp
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import random
sns.set_style('whitegrid')
import itertools
import json
import math

from math import factorial
from IPython.core.display import display, HTML

In [2]:
df = pd.read_csv('../../src/clean_csv/clean_dataframe.csv', sep='\t')
df.columns = ['index', 'id_par','label', 'statement', 'subject', 'speaker', 
              'job', 'state', 'party', 'context']
df = df.drop(['index'], axis=1)

df = df[~df['id_par'].isin(['1779.json'])]
# replace string
t_repl = {'barely-true': "BARELYTRUE",
          'false': "FALSE",
          'half-true': "HALFTRUE",
          'mostly-true': "MOSTLYTRUE",
          'pants-fire' : "LIE",
          'true' : "TRUE"
        }
party_repl = {'democrat' : "DEM",
              'republican' : "REP"
            }

def f1(l):
    return [ t_repl.get(item,item) for item in l ]
def f2(l):
    return [ party_repl.get(item,item) for item in l ]
df['label'] = df.label.str.split('<').apply(f1).str.join('')
df['party'] = df.party.str.split('<').apply(f2).str.join('')

df['name'] = '-1'
display(df.head(5))
display(df.shape[0])

,id_par,label,statement,subject,speaker,job,state,party,context,name
0,10540.json,HALFTRUE,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,DEM,a floor speech.,-1
1,324.json,MOSTLYTRUE,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,DEM,Denver,-1
2,12465.json,TRUE,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,REP,a an online opinion-piece,-1
3,153.json,HALFTRUE,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,DEM,"a Democratic debate in Philadelphia, Pa.",-1
4,9741.json,MOSTLYTRUE,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,REP,an online video,-1


8433

In [3]:
# Prendo solo Democratici, Repubblicani e Bernie
sub_df = df[(df.party == "DEM") | (df.party == "REP")]

sub_df['speaker'] = sub_df['speaker'].str.replace('-',' ')
sub_df['speaker'] = sub_df.speaker.str.title()

display(sub_df.tail())
display(sub_df.shape)

c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,id_par,label,statement,subject,speaker,job,state,party,context,name
8428,7285.json,LIE,Says Chris Christies plan to kick-start our ec...,taxes,Barbara Buono,State Senator,New Jersey,DEM,an interview on bluejersey.com,-1
8429,9436.json,BARELYTRUE,We came out of the White House not only dead b...,"candidates-biography,elections",Hillary Clinton,Presidential candidate,New York,DEM,an interview on ABC,-1
8430,7334.json,HALFTRUE,Says his budget provides the highest state fun...,education,Rick Scott,Governor,Florida,REP,a news conference,-1
8431,9788.json,BARELYTRUE,Ive been here almost every day.,"civil-rights,crime,criminal-justice",Jay Nixon,Governor,Missouri,DEM,"on ABC's ""This Week""",-1
8433,3186.json,BARELYTRUE,Says an EPA permit languished under Strickland...,"environment,government-efficiency",John Kasich,"Governor of Ohio as of Jan. 10, 2011",Ohio,REP,a news conference,-1


(7546, 10)

In [4]:
filtered = pd.DataFrame(columns = sub_df.columns)
categories = ["LIE", "FALSE", "BARELYTRUE", "HALFTRUE", "MOSTLYTRUE", "TRUE"]
par = ["DEM", "REP"]
docs_per_cat = 20
docs_per_cat_by_party = int(docs_per_cat/len(par))

for cat in categories:
    for p in par:
        c = sub_df['label'] == cat
        p = sub_df['party'] == p
        # Prendo i primi docs_per_cat per ogni label
        sub = sub_df[c & p]
        
        sub = sub.head(docs_per_cat_by_party)
        assert sub.shape[0] == docs_per_cat/2, 'Non ho abbastanza docs per tutte le categorie'
        # Nomino i documenti
        sub['name'] = [cat + '_doc' + str(x + 1) for x in range(len(sub))]
        # Concateno il tutto
        filtered  = pd.concat([filtered, sub])

filtered["name"] = filtered["party"].map(str) + '_' +  filtered["name"]
# Compongo il Json con i campi necessari
filtered["js"] = [ {"id_par": filtered.iloc[x]['id_par'],
                    "name": filtered.iloc[x]['name'],
                    "statement": (filtered.iloc[x]['statement']).replace("'", ""),
                    "speaker": filtered.iloc[x]['speaker'],
                    "job": filtered.iloc[x]['job'],
                    "context": filtered.iloc[x]['context'],
                    "party": filtered.iloc[x]['party'],
                    }
                  for x in range(len(filtered))]
display(filtered.groupby(['party']).count())
filtered = filtered[['name', 'js']]

# Aggiungo Gold High | Low
high = {'id_par':'HIGH',
        'name':'HIGH',
        'statement':'Yes, We can!',
        'speaker':'Barack Obama',
        'job':'Former U.S. President',
        'context':'Public appearance',
        'party':'DEM',
        }
low = {'id_par':'LOW',
        'name':'LOW',
        'statement':'If I were not a Formula One pilot, I would be a football player.',
        'speaker':'Abraham Lincoln',
        'job':'Former U.S. President',
        'context':'Public appearance',
        'party':'REP'
        }

filtered = filtered.append({'name' : 'HIGH', 'js': high}, ignore_index = True)
filtered = filtered.append({'name' : 'LOW', 'js': low}, ignore_index = True)

display(filtered.tail())

,id_par,label,statement,subject,speaker,job,state,context,name,js
party,,,,,,,,,,
DEM,60,60,60,60,60,60,60,60,60,60
REP,60,60,60,60,60,60,60,60,60,60


,name,js
117,REP_TRUE_doc8,"{'id_par': '4681.json', 'name': 'REP_TRUE_doc8..."
118,REP_TRUE_doc9,"{'id_par': '5044.json', 'name': 'REP_TRUE_doc9..."
119,REP_TRUE_doc10,"{'id_par': '7261.json', 'name': 'REP_TRUE_doc1..."
120,HIGH,"{'id_par': 'HIGH', 'name': 'HIGH', 'statement'..."
121,LOW,"{'id_par': 'LOW', 'name': 'LOW', 'statement': ..."


In [5]:
categories = ['LIE', 'FALSE', 'BARELYTRUE', 'HALFTRUE', 'MOSTLYTRUE', 'TRUE']
fst_cat = ['LIE', 'FALSE', 'BARELYTRUE']
lst_cat = ['HALFTRUE', 'MOSTLYTRUE', 'TRUE']
par = ['DEM', 'REP']
total_docs = docs_per_cat * len(categories)
docs_each_hit = 6
gold_each_hit = 2
repetitions_each_doc = 10
actual_docs_each_hit = docs_each_hit + gold_each_hit
assert total_docs % docs_each_hit == 0, 'Errore1'
assert docs_each_hit % len(categories) == 0, 'Errore2'

# Shuffle della riga o rifaccio campione dall'inizio, per ora shuffle della riga.
df = pd.DataFrame(columns = ['d{}'.format(x + 1) for x in range(actual_docs_each_hit)])
pool_rep = {"{}".format(categories[j]):["REP_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}
pool_dem = {"{}".format(categories[j]):["DEM_{}_doc{}".format(categories[j], i + 1) 
                                        for i in range(docs_per_cat_by_party)] for j in range(len(categories))}

def rnd_pick(sample, pool, pick, length):
    for i in range(length):
        value = random.choice(pool[pick[i]])
        sample.append(value)
        lst = pool[pick[i]]
        lst.remove(value)
        pool[pick[i]] = lst
    return sample

def single_hit(pool_rep, pool_dem, categories, docs_per_cat_by_party):
    sample1 = []
    sample2 = []
    pick_dem1 = random.sample(categories, 3)
    pick_rep2 = pick_dem1
    # Rimuovo dalle opzioni le cat dei dem
    pick_rep1 = list(set(categories) - set(pick_dem1))
    pick_dem2 = pick_rep1
    # Prendo 3 Dem
    sample1 = rnd_pick(sample1, pool_dem, pick_dem1, 3)
    # Prendo 3 Rep
    sample1 = rnd_pick(sample1, pool_rep, pick_rep1, 3)
    sample1.append('HIGH')
    sample1.append('LOW')
    sample2 = rnd_pick(sample2, pool_dem, pick_dem2, 3)
    # Prendo 3 Rep
    sample2 = rnd_pick(sample2, pool_rep, pick_rep2, 3)
    sample2.append('HIGH')
    sample2.append('LOW')
    return sample1, sample2

#sample1, sample2 = single_hit(pool_rep, pool_dem, categories, docs_per_cat_by_party)

for i in range( int((total_docs / docs_each_hit)/2) ):
    hit_sample = []
    choice1, choice2 = single_hit(pool_rep, pool_dem, categories, docs_per_cat_by_party)
    hit_sample.append(choice1)
    hit_sample.append(choice2)
    # repeat the hits j times
    for j in range(repetitions_each_doc):
        random.shuffle(hit_sample[0])
        random.shuffle(hit_sample[1])
        df.loc[len(df)] = hit_sample[0]
        df.loc[len(df)] = hit_sample[1]
        
display(df.tail())
display(filtered.tail())
display('HITs: {}'.format(df.shape[0]))
display( 'judgments:{}'.format(df.shape[0] * docs_each_hit) )
price = 0.5
display( 'cost:{}'.format(df.shape[0] * price) )

,d1,d2,d3,d4,d5,d6,d7,d8
195,DEM_HALFTRUE_doc2,DEM_BARELYTRUE_doc1,REP_TRUE_doc4,LOW,REP_MOSTLYTRUE_doc7,REP_LIE_doc2,DEM_FALSE_doc8,HIGH
196,LOW,DEM_LIE_doc3,REP_BARELYTRUE_doc3,HIGH,DEM_MOSTLYTRUE_doc9,REP_HALFTRUE_doc1,REP_FALSE_doc9,DEM_TRUE_doc9
197,REP_LIE_doc2,REP_MOSTLYTRUE_doc7,DEM_FALSE_doc8,REP_TRUE_doc4,LOW,DEM_HALFTRUE_doc2,DEM_BARELYTRUE_doc1,HIGH
198,HIGH,REP_FALSE_doc9,DEM_TRUE_doc9,REP_BARELYTRUE_doc3,DEM_MOSTLYTRUE_doc9,DEM_LIE_doc3,REP_HALFTRUE_doc1,LOW
199,DEM_HALFTRUE_doc2,DEM_BARELYTRUE_doc1,DEM_FALSE_doc8,HIGH,REP_TRUE_doc4,REP_LIE_doc2,LOW,REP_MOSTLYTRUE_doc7


,name,js
117,REP_TRUE_doc8,"{'id_par': '4681.json', 'name': 'REP_TRUE_doc8..."
118,REP_TRUE_doc9,"{'id_par': '5044.json', 'name': 'REP_TRUE_doc9..."
119,REP_TRUE_doc10,"{'id_par': '7261.json', 'name': 'REP_TRUE_doc1..."
120,HIGH,"{'id_par': 'HIGH', 'name': 'HIGH', 'statement'..."
121,LOW,"{'id_par': 'LOW', 'name': 'LOW', 'statement': ..."


'HITs: 200'

'judgments:1200'

'cost:100.0'

In [6]:
units = df.copy()

keys = filtered['name'].values
vals = filtered['js'].values
testo_dict = {keys[i]:vals[i] for i in range(len(filtered))}
# Mappo
for col in units.columns.values:
    units[col] = [testo_dict.get(item,item) for item in units[col] ]
    if(str(units[col]) == "HIGH"):
        display(units[col])
    
# Salvo posizione High e Low
highs = []
lows = []
for i in range(len(units)):
    sub = list(df.iloc[i].values)
    h = sub.index('HIGH') + 1
    l = sub.index('LOW') + 1
    highs.append(h)
    lows.append(l)
# Aggiunta a DataFrame
df['high'] = highs
df['low'] = lows
df['id_unit'] = ['unit_' + str(x) for x in range(len(units))]

display(units.head())
display(filtered.head())
display(df.head())

df.to_csv('../../Units/Task/SCHEMAunits.csv', index = False, header = True)

,d1,d2,d3,d4,d5,d6,d7,d8
0,"{'id_par': '1659.json', 'name': 'REP_LIE_doc6'...","{'id_par': '10215.json', 'name': 'DEM_FALSE_do...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '12372.json', 'name': 'DEM_HALFTRUE...","{'id_par': '13237.json', 'name': 'DEM_TRUE_doc...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '321.json', 'name': 'REP_MOSTLYTRUE...","{'id_par': '12981.json', 'name': 'REP_BARELYTR..."
1,"{'id_par': '4681.json', 'name': 'REP_TRUE_doc8...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '10173.json', 'name': 'REP_FALSE_do...","{'id_par': '11513.json', 'name': 'DEM_BARELYTR...","{'id_par': '534.json', 'name': 'REP_HALFTRUE_d...","{'id_par': '12385.json', 'name': 'DEM_MOSTLYTR...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '9280.json', 'name': 'DEM_LIE_doc8'..."
2,"{'id_par': '12372.json', 'name': 'DEM_HALFTRUE...","{'id_par': '13237.json', 'name': 'DEM_TRUE_doc...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '10215.json', 'name': 'DEM_FALSE_do...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '1659.json', 'name': 'REP_LIE_doc6'...","{'id_par': '12981.json', 'name': 'REP_BARELYTR...","{'id_par': '321.json', 'name': 'REP_MOSTLYTRUE..."
3,"{'id_par': '11513.json', 'name': 'DEM_BARELYTR...","{'id_par': '4681.json', 'name': 'REP_TRUE_doc8...","{'id_par': '534.json', 'name': 'REP_HALFTRUE_d...","{'id_par': '9280.json', 'name': 'DEM_LIE_doc8'...","{'id_par': '12385.json', 'name': 'DEM_MOSTLYTR...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '10173.json', 'name': 'REP_FALSE_do...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ..."
4,"{'id_par': '1659.json', 'name': 'REP_LIE_doc6'...","{'id_par': '13237.json', 'name': 'DEM_TRUE_doc...","{'id_par': '12981.json', 'name': 'REP_BARELYTR...","{'id_par': 'HIGH', 'name': 'HIGH', 'statement'...","{'id_par': '10215.json', 'name': 'DEM_FALSE_do...","{'id_par': '12372.json', 'name': 'DEM_HALFTRUE...","{'id_par': 'LOW', 'name': 'LOW', 'statement': ...","{'id_par': '321.json', 'name': 'REP_MOSTLYTRUE..."


,name,js
0,DEM_LIE_doc1,"{'id_par': '10829.json', 'name': 'DEM_LIE_doc1..."
1,DEM_LIE_doc2,"{'id_par': '162.json', 'name': 'DEM_LIE_doc2',..."
2,DEM_LIE_doc3,"{'id_par': '9969.json', 'name': 'DEM_LIE_doc3'..."
3,DEM_LIE_doc4,"{'id_par': '8510.json', 'name': 'DEM_LIE_doc4'..."
4,DEM_LIE_doc5,"{'id_par': '1044.json', 'name': 'DEM_LIE_doc5'..."


,d1,d2,d3,d4,d5,d6,d7,d8,high,low,id_unit
0,REP_LIE_doc6,DEM_FALSE_doc1,LOW,DEM_HALFTRUE_doc3,DEM_TRUE_doc2,HIGH,REP_MOSTLYTRUE_doc5,REP_BARELYTRUE_doc4,6,3,unit_0
1,REP_TRUE_doc8,LOW,REP_FALSE_doc2,DEM_BARELYTRUE_doc9,REP_HALFTRUE_doc10,DEM_MOSTLYTRUE_doc3,HIGH,DEM_LIE_doc8,7,2,unit_1
2,DEM_HALFTRUE_doc3,DEM_TRUE_doc2,HIGH,DEM_FALSE_doc1,LOW,REP_LIE_doc6,REP_BARELYTRUE_doc4,REP_MOSTLYTRUE_doc5,3,5,unit_2
3,DEM_BARELYTRUE_doc9,REP_TRUE_doc8,REP_HALFTRUE_doc10,DEM_LIE_doc8,DEM_MOSTLYTRUE_doc3,HIGH,REP_FALSE_doc2,LOW,6,8,unit_3
4,REP_LIE_doc6,DEM_TRUE_doc2,REP_BARELYTRUE_doc4,HIGH,DEM_FALSE_doc1,DEM_HALFTRUE_doc3,LOW,REP_MOSTLYTRUE_doc5,4,7,unit_4


In [7]:
## Genero i docs
sep = '=++='

## Salvo in .txt High e Low
id_par = 'HIGH'
name = 'HIGH'
statement = 'Yes, We can!'
speaker = 'Barack Obama'
job = 'Former U.S. President'
context = 'Public appearance'
party = 'DEM'
my_str = id_par + sep + name + sep + statement + sep + speaker + sep + job + sep + context + sep + party

text_file = open("../../Units/Task/remote-units/{}.txt".format(name), "w")
text_file.write(my_str)
text_file.close()
id_par = 'LOW'
name = 'LOW'
statement = 'If I were not a Formula One pilot, I would be a football player.'
speaker = 'Abraham Lincoln'
job = 'Former U.S. President'
context ='Public appearance'
party = 'REP'
my_str = id_par + sep + name + sep + statement + sep + speaker + sep + job + sep + context + sep + party

text_file = open("../../Units/Task/remote-units/{}.txt".format(name), "w")
text_file.write(my_str)
text_file.close()

## Other Statements
for unit in range(len(units)):
    print('{}, '.format(unit), end = '')
    for doc in range(1, 9):
        
        # id_par, name, statement, speaker, job, context
        dic = units.loc[unit]["d" + str(doc)]
        id_par = dic['id_par']
        name = dic['name']
        statement = dic['statement']
        speaker = dic['speaker']
        job = str(dic['job'])
        context = str(dic['context'])
        party = dic['party']
        
        sep = "=++="
        my_str = id_par + sep + name + sep + statement + sep + speaker + sep + job + sep + context + sep + party
        #display(my_str)
        text_file = open("../../Units/Task/remote-units/{}.txt".format(name), "w")
        text_file.write(my_str)
        text_file.close()

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 